In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
import warnings
warnings.filterwarnings("ignore")

In [22]:
# display
pasture_type = 'clover_rye_grass'

In [23]:
read_file = pasture_type+'_data.csv'
df = pd.read_csv(read_file)
df = df.drop(['RAIN', 'MAX_TEMP', 'MIN_TEMP', 'RH_TMAX',
       'RH_TMIN', 'EVAP', 'RADIATION'], axis = 1)

In [24]:
df

,3DAVG_RAIN,3DAVG_MAX_TEMP,3DAVG_MIN_TEMP,3DAVG_RH_TMAX,3DAVG_RH_TMIN,3DAVG_EVAP,3DAVG_RADIATION,3D_MIN_TEMP,3D_MAX_TEMP,5DAVG_RAIN,...,365DAVG_RAIN,365DAVG_MAX_TEMP,365DAVG_MIN_TEMP,365DAVG_RH_TMAX,365DAVG_RH_TMIN,365DAVG_EVAP,365DAVG_RADIATION,365D_MIN_TEMP,365D_MAX_TEMP,FOO
0,0.000000,27.500000,12.150000,40.750000,98.050000,7.850000,26.250000,11.5,32.0,0.000,...,0.000000,27.500000,12.150000,40.750000,98.050000,7.850000,26.250000,11.5,32.0,396.0
1,0.100000,22.166667,7.400000,40.500000,97.900000,6.500000,25.733333,2.6,24.3,0.075,...,0.075000,24.625000,8.750000,37.850000,97.450000,6.850000,27.125000,2.6,32.0,393.0
2,0.000000,34.000000,10.066667,19.700000,85.066667,7.233333,33.333333,8.2,35.1,0.060,...,0.042857,28.642857,9.314286,30.071429,92.142857,7.014286,29.785714,2.6,35.1,379.0
3,0.000000,29.900000,12.233333,29.366667,84.400000,7.866667,28.833333,11.2,35.1,0.000,...,0.033333,28.344444,10.077778,30.922222,90.188889,7.255556,29.133333,2.6,35.1,379.0
4,0.000000,31.466667,11.333333,24.533333,83.966667,7.100000,30.533333,9.3,33.5,0.000,...,0.025000,29.125000,10.391667,29.325000,88.633333,7.216667,29.483333,2.6,35.1,377.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16740,0.000000,24.633333,9.433333,37.800000,93.833333,5.633333,23.333333,5.9,26.5,0.000,...,2.028219,21.766575,9.186575,49.775068,94.352055,4.037260,14.572877,0.5,39.1,2838.0
16741,1.366667,22.900000,9.766667,48.333333,93.833333,5.633333,23.433333,4.5,27.1,0.820,...,2.039452,21.731781,9.175616,49.886849,94.382740,4.023836,14.544384,0.5,39.1,2841.0
16742,0.000000,32.600000,12.766667,27.633333,92.100000,7.133333,31.500000,12.4,33.5,0.000,...,2.039452,21.728219,9.181370,49.944932,94.493973,4.020274,14.555890,0.5,39.1,2793.0
16743,0.000000,36.600000,16.333333,22.766667,76.100000,12.066667,28.833333,14.7,38.5,0.000,...,2.039452,21.797808,9.207671,49.863014,94.441096,4.047671,14.606849,0.5,39.1,2774.0


In [4]:
y = df['FOO']
X = df.drop(['FOO'], axis=1)

In [5]:
X = pd.get_dummies(X)

In [6]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the target variable
scaler = StandardScaler()
train_target = scaler.fit_transform(y_train.values.reshape(-1, 1))
test_target = scaler.transform(y_test.values.reshape(-1, 1))

In [7]:
def eval(scaler_targ,model_in, X_train,y_train,X_test,y_test):
    
    y_train = scaler_targ.inverse_transform(y_train)
    y_test = scaler_targ.inverse_transform(y_test)
     
    # Predict the labels of the train and test sets: preds
    preds_train = scaler_targ.inverse_transform(model_in.predict(X_train).reshape(-1, 1))
    preds_test = scaler_targ.inverse_transform(model_in.predict(X_test).reshape(-1, 1))
    
    # compute the rmse: rmse
    rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, preds_test))
    
    # compute the mae: mae
    mae_train = mean_absolute_error(y_train,preds_train)
    mae_test = mean_absolute_error(y_test, preds_test)
    
     # compute the r2 score
    r2_train = r2_score(y_train,preds_train)
    r2_test = r2_score(y_test,preds_test)
    
    print('*********Training*********')
    print("\tRMSE: %0.2f" % (rmse_train))
    print("\tMAE: %0.2f" % (mae_train))
    print("\tR2 score: %0.2f" % (r2_train))


    print('*********Testing*********')
    print("\tRMSE: %0.2f" % (rmse_test))
    print("\tMAE: %0.2f" % (mae_test))
    print("\tR2 score: %0.2f" % (r2_test))

    return preds_train,preds_test,rmse_train,rmse_test,mae_train,mae_test,r2_train,r2_test

### Climate only

In [10]:
days_in_out = ['3D','5D', '7D', '10D', '14D', '30D', '90D', '180D', '365D']
XGB_rmse_all_train = ['XGBoost']
bootstra_rmse_all_train = ['Bootstrap']
RF_rmse_all_train = ['Random Forest']
LR_rmse_all_train = ['Linear Regression']
DNN_rmse_all_train = ['DNN']
stack_xgb_dnn_rmse_all_train = ['Stacking XGB DNN']
stack_xgb_rf_rmse_all_train = ['Stacking XGB RF']

XGB_mae_all_train = ['XGBoost']
bootstra_mae_all_train = ['Bootstrap']
RF_mae_all_train = ['Random Forest']
LR_mae_all_train = ['Linear Regression']
DNN_mae_all_train = ['DNN']
stack_xgb_dnn_mae_all_train = ['Stacking XGB DNN']
stack_xgb_rf_mae_all_train = ['Stacking XGB RF']

XGB_r2_all_train = ['XGBoost']
bootstra_r2_all_train = ['Bootstrap']
RF_r2_all_train = ['Random Forest']
LR_r2_all_train = ['Linear Regression']
DNN_r2_all_train = ['DNN']
stack_xgb_dnn_r2_all_train = ['Stacking XGB DNN']
stack_xgb_rf_r2_all_train = ['Stacking XGB RF']


XGB_rmse_all_test = ['XGBoost']
bootstra_rmse_all_test = ['Bootstrap']
RF_rmse_all_test = ['Random Forest']
LR_rmse_all_test = ['Linear Regression']
DNN_rmse_all_test = ['DNN']
stack_xgb_dnn_rmse_all_test = ['Stacking XGB DNN']
stack_xgb_rf_rmse_all_test = ['Stacking XGB RF']

XGB_mae_all_test = ['XGBoost']
bootstra_mae_all_test = ['Bootstrap']
RF_mae_all_test = ['Random Forest']
LR_mae_all_test = ['Linear Regression']
DNN_mae_all_test = ['DNN']
stack_xgb_dnn_mae_all_test = ['Stacking XGB DNN']
stack_xgb_rf_mae_all_test = ['Stacking XGB RF']

XGB_r2_all_test = ['XGBoost']
bootstra_r2_all_test = ['Bootstrap']
RF_r2_all_test = ['Random Forest']
LR_r2_all_test = ['Linear Regression']
DNN_r2_all_test = ['DNN']
stack_xgb_dnn_r2_all_test = ['Stacking XGB DNN']
stack_xgb_rf_r2_all_test = ['Stacking XGB RF']


for i in range(0,len(days_in_out)):
    days_in = days_in_out.copy()
    to_include = days_in.pop(i)
    result_string = '|'.join(days_in)
    #print(result_string)
    print(to_include)
    if to_include == '365D':
        result_string =  '^(?!365D).'
    X_train_climate = X_train.copy()
    X_test_climate = X_test.copy()
    #print(X_train_climate.columns)
    X_train_climate.drop(X_train_climate.filter(regex=result_string).columns, axis=1, inplace=True)
    #print(X_train_climate.columns)
    X_train_climate.drop(X_train_climate.filter(regex=to_include+'_').columns, axis=1, inplace=True)
    X_test_climate.drop(X_test_climate.filter(regex=result_string).columns, axis=1, inplace=True)
    X_test_climate.drop(X_test_climate.filter(regex=to_include+'_').columns, axis=1, inplace=True)
    #print('Climate attributes: ',X_train_climate.columns)
    #print('**********Climate only************')
    
    scaler2 = StandardScaler()
    X_train_climate = scaler2.fit_transform(X_train_climate)
    X_test_climate = scaler2.transform(X_test_climate)
    
    print('XGBoost')
    # Instantiatethe XGBRegressor: xg_reg
    xg_climate = xgb.XGBRegressor(objective='reg:squarederror', seed=42, n_estimators=30, max_depth=5, reg_lambda = 1).fit(X_train_climate, train_target)
    XG_C_preds_train,XG_C_preds_test,XG_C_rmse_train,XG_C_rmse_test,XG_C_mae_train,XG_C_mae_test,XG_C_r2_train,XG_C_r2_test = eval(scaler, xg_climate, X_train_climate,train_target,X_test_climate,test_target)
    XGB_rmse_all_train.append(XG_C_rmse_train)
    XGB_rmse_all_test.append(XG_C_rmse_test)

    XGB_mae_all_train.append(XG_C_mae_train)
    XGB_mae_all_test.append(XG_C_mae_test)
    
    XGB_r2_all_train.append(XG_C_r2_train)
    XGB_r2_all_test.append(XG_C_r2_test)
    
    from sklearn.ensemble import BaggingRegressor
    print('Bootstrapped Aggregating (Bagging)')
    base_esti = xgb.XGBRegressor(objective='reg:squarederror', seed=42, n_estimators=30, max_depth=5, reg_lambda = 1)
    n_estis = 50
    bagging_model = BaggingRegressor(estimator=base_esti, n_estimators=n_estis, random_state=42)
    bagging_model.fit(X_train_climate, train_target)
    BXG_C_preds_train,BXG_C_preds_test,BXG_C_rmse_train,BXG_C_rmse_test,BXG_C_mae_train,BXG_C_mae_test,BXG_C_r2_train,BXG_C_r2_test = eval(scaler, bagging_model,  X_train_climate,train_target,X_test_climate,test_target)
    bootstra_rmse_all_train.append(BXG_C_rmse_train)
    bootstra_rmse_all_test.append(BXG_C_rmse_test)
    bootstra_mae_all_train.append(BXG_C_mae_train)
    bootstra_mae_all_test.append(BXG_C_mae_test)
    bootstra_r2_all_train.append(BXG_C_r2_train)
    bootstra_r2_all_test.append(BXG_C_r2_test)

    from sklearn.ensemble import RandomForestRegressor
    print('Random Forest')
    # Instantiatethe XGBRegressor: xg_reg
    rf_climate = RandomForestRegressor(max_depth=5, random_state = 5).fit(X_train_climate, train_target)
    rf_C_preds_train,rf_C_preds_test,rf_C_rmse_train,rf_C_rmse_test,rf_C_mae_train,rf_C_mae_test,rf_C_r2_train,rf_C_r2_test = eval(scaler, rf_climate,  X_train_climate,train_target,X_test_climate,test_target)
    RF_rmse_all_train.append(rf_C_rmse_train)
    RF_rmse_all_test.append(rf_C_rmse_test)
    RF_mae_all_train.append(rf_C_mae_train)
    RF_mae_all_test.append(rf_C_mae_test)
    RF_r2_all_train.append(rf_C_r2_train)
    RF_r2_all_test.append(rf_C_r2_test)
    
    
    from sklearn.linear_model import LinearRegression
    print('Linear Regression')
    lr_climate = LinearRegression().fit(X_train_climate, train_target)
    lr_C_preds_train,lr_C_preds_test,lr_C_rmse_train,lr_C_rmse_test,lr_C_mae_train,lr_C_mae_test,lr_C_r2_train,lr_C_r2_test = eval(scaler, lr_climate,  X_train_climate,train_target,X_test_climate,test_target)
    LR_rmse_all_train.append(lr_C_rmse_train)
    LR_rmse_all_test.append(lr_C_rmse_test)
    LR_mae_all_train.append(lr_C_mae_train)
    LR_mae_all_test.append(lr_C_mae_test)
    LR_r2_all_train.append(lr_C_r2_train)
    LR_r2_all_test.append(lr_C_r2_test)
    
    
    print('DNN')
    dnn_climate = MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes=(75,75)).fit(X_train_climate, np.array(train_target).reshape(-1, ))
    DNN_C_preds_train,DNN_C_preds_test,DNN_C_rmse_train,DNN_C_rmse_test,DNN_C_mae_train,DNN_C_mae_test,DNN_C_r2_train,DNN_C_r2_test = eval(scaler, dnn_climate, X_train_climate,train_target,X_test_climate,test_target)
    DNN_rmse_all_train.append(DNN_C_rmse_train)
    DNN_rmse_all_test.append(DNN_C_rmse_test)
    DNN_mae_all_train.append(DNN_C_mae_train)
    DNN_mae_all_test.append(DNN_C_mae_test)
    DNN_r2_all_train.append(DNN_C_r2_train)
    DNN_r2_all_test.append(DNN_C_r2_test)


    print('Stacking XGBoost + DNN')
    from sklearn.linear_model import LinearRegression
    from sklearn.ensemble import StackingRegressor
    base_models = [
        ('XGBoost', xgb.XGBRegressor(objective='reg:squarederror', seed=42, n_estimators=30, max_depth=5, reg_lambda = 1)),
        ('DNN', MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes=(75,75)))
    ]
    # Define the meta-model (final estimator)
    meta_model = LinearRegression()
    # Create the stacking model
    stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model)
    # Train the stacking model
    stacking_regressor.fit(X_train_climate, train_target)
    stack_C_preds_train,stack_C_preds_test,stack_C_rmse_train,stack_C_rmse_test,stack_C_mae_train,stack_C_mae_test,stack_C_r2_train,stack_C_r2_test = eval(scaler,stacking_regressor, X_train_climate,train_target,X_test_climate,test_target)
    
    stack_xgb_dnn_rmse_all_train.append(stack_C_rmse_train)
    stack_xgb_dnn_rmse_all_test.append(stack_C_rmse_test)
    stack_xgb_dnn_mae_all_train.append(stack_C_mae_train)
    stack_xgb_dnn_mae_all_test.append(stack_C_mae_test)
    stack_xgb_dnn_r2_all_train.append(stack_C_r2_train)
    stack_xgb_dnn_r2_all_test.append(stack_C_r2_test)
    
    print('Stacking XGBoost + RF')
    base_models_xr = [
        ('XGBoost', xgb.XGBRegressor(objective='reg:squarederror', seed=42, n_estimators=30, max_depth=5, reg_lambda = 1)),
        ('RF', RandomForestRegressor(max_depth=5, random_state = 5))
    ]
    # Define the meta-model (final estimator)
    meta_model_xr = MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes=(75,75))
    # Create the stacking model
    stacking_regressor_xr = StackingRegressor(estimators=base_models_xr, final_estimator=meta_model_xr)
    # Train the stacking model
    stacking_regressor_xr.fit(X_train_climate, train_target)
    stack_xr_C_preds_train,stack_xr_C_preds_test,stack_xr_C_rmse_train,stack_xr_C_rmse_test,stack_xr_C_mae_train,stack_xr_C_mae_test,stack_xr_C_r2_train,stack_xr_C_r2_test = eval(scaler,stacking_regressor_xr, X_train_climate,train_target,X_test_climate,test_target)
    stack_xgb_rf_rmse_all_train.append(stack_xr_C_rmse_train)
    stack_xgb_rf_rmse_all_test.append(stack_xr_C_rmse_test)
    stack_xgb_rf_mae_all_train.append(stack_xr_C_mae_train)
    stack_xgb_rf_mae_all_test.append(stack_xr_C_mae_test)
    stack_xgb_rf_r2_all_train.append(stack_xr_C_r2_train)
    stack_xgb_rf_r2_all_test.append(stack_xr_C_r2_test)


3D
XGBoost
*********Training*********
	RMSE: 297.53
	MAE: 181.31
	R2 score: 0.92
*********Testing*********
	RMSE: 297.93
	MAE: 184.70
	R2 score: 0.92
Bootstrapped Aggregating (Bagging)
*********Training*********
	RMSE: 295.99
	MAE: 178.67
	R2 score: 0.92
*********Testing*********
	RMSE: 296.52
	MAE: 182.48
	R2 score: 0.92
Random Forest
*********Training*********
	RMSE: 487.53
	MAE: 307.39
	R2 score: 0.78
*********Testing*********
	RMSE: 476.58
	MAE: 309.39
	R2 score: 0.79
Linear Regression
*********Training*********
	RMSE: 843.88
	MAE: 648.22
	R2 score: 0.35
*********Testing*********
	RMSE: 836.50
	MAE: 651.02
	R2 score: 0.36
DNN
*********Training*********
	RMSE: 299.74
	MAE: 182.63
	R2 score: 0.92
*********Testing*********
	RMSE: 300.64
	MAE: 186.46
	R2 score: 0.92
Stacking XGBoost + DNN
*********Training*********
	RMSE: 295.71
	MAE: 179.05
	R2 score: 0.92
*********Testing*********
	RMSE: 296.24
	MAE: 182.34
	R2 score: 0.92
Stacking XGBoost + RF
*********Training*********
	RMSE: 295.0

*********Training*********
	RMSE: 313.60
	MAE: 185.52
	R2 score: 0.91
*********Testing*********
	RMSE: 308.21
	MAE: 185.63
	R2 score: 0.91
Stacking XGBoost + DNN
*********Training*********
	RMSE: 291.94
	MAE: 173.74
	R2 score: 0.92
*********Testing*********
	RMSE: 293.16
	MAE: 177.38
	R2 score: 0.92
Stacking XGBoost + RF
*********Training*********
	RMSE: 296.96
	MAE: 176.72
	R2 score: 0.92
*********Testing*********
	RMSE: 298.69
	MAE: 179.62
	R2 score: 0.92
365D
XGBoost
*********Training*********
	RMSE: 291.54
	MAE: 173.55
	R2 score: 0.92
*********Testing*********
	RMSE: 293.20
	MAE: 177.37
	R2 score: 0.92
Bootstrapped Aggregating (Bagging)
*********Training*********
	RMSE: 291.59
	MAE: 173.42
	R2 score: 0.92
*********Testing*********
	RMSE: 293.19
	MAE: 177.19
	R2 score: 0.92
Random Forest
*********Training*********
	RMSE: 301.34
	MAE: 184.12
	R2 score: 0.92
*********Testing*********
	RMSE: 301.49
	MAE: 186.19
	R2 score: 0.92
Linear Regression
*********Training*********
	RMSE: 494.92


In [11]:

import pandas as pd
 
# List
df_RMSE_training = [XGB_rmse_all_train,bootstra_rmse_all_train,RF_rmse_all_train,LR_rmse_all_train,DNN_rmse_all_train,stack_xgb_dnn_rmse_all_train,stack_xgb_rf_rmse_all_train]
df_MAE_training = [XGB_mae_all_train,bootstra_mae_all_train,RF_mae_all_train,LR_mae_all_train,DNN_mae_all_train,stack_xgb_dnn_mae_all_train,stack_xgb_rf_mae_all_train]
df_R2_training = [XGB_r2_all_train,bootstra_r2_all_train,RF_r2_all_train,LR_r2_all_train,DNN_r2_all_train,stack_xgb_dnn_r2_all_train,stack_xgb_rf_r2_all_train]


df_RMSE_test = [XGB_rmse_all_test,bootstra_rmse_all_test,RF_rmse_all_test,LR_rmse_all_test,DNN_rmse_all_test,stack_xgb_dnn_rmse_all_test,stack_xgb_rf_rmse_all_test]
df_MAE_test = [XGB_mae_all_test,bootstra_mae_all_test,RF_mae_all_test,LR_mae_all_test,DNN_mae_all_test,stack_xgb_dnn_mae_all_test,stack_xgb_rf_mae_all_test]
df_R2_test = [XGB_r2_all_test,bootstra_r2_all_test,RF_r2_all_test,LR_r2_all_test,DNN_r2_all_test,stack_xgb_dnn_r2_all_test,stack_xgb_rf_r2_all_test]
 
#Create a DataFrame object
df_RMSE_training_output = pd.DataFrame(df_RMSE_training, 
                  columns = ['Model','3D', '5D', '7D', '10D', '14D', '30D', '90D', '180D', '365D']) 
df_RMSE_test_output = pd.DataFrame(df_RMSE_test, 
                  columns = ['Model','3D', '5D', '7D', '10D', '14D', '30D', '90D', '180D', '365D']) 
              
df_MAE_training_output = pd.DataFrame(df_MAE_training, 
                  columns = ['Model','3D', '5D', '7D', '10D', '14D', '30D', '90D', '180D', '365D']) 
df_MAE_test_output = pd.DataFrame(df_MAE_test, 
                  columns = ['Model','3D', '5D', '7D', '10D', '14D', '30D', '90D', '180D', '365D']) 
              
df_R2_training_output = pd.DataFrame(df_R2_training, 
                  columns = ['Model','3D', '5D', '7D', '10D', '14D', '30D', '90D', '180D', '365D']) 
df_R2_test_output = pd.DataFrame(df_R2_test, 
                  columns = ['Model','3D', '5D', '7D', '10D', '14D', '30D', '90D', '180D', '365D']) 
 

print('RMSE')
display(df_RMSE_training_output)
df_RMSE_training_output.to_csv('rmse_training_'+pasture_type+'.csv', index = False)
display(df_RMSE_test_output)
df_RMSE_test_output.to_csv('rmse_testing_'+pasture_type+'.csv', index = False)
print('MAE')
display(df_MAE_training_output)
df_MAE_training_output.to_csv('mae_training_'+pasture_type+'.csv', index = False)
display(df_MAE_test_output)
df_MAE_test_output.to_csv('mae_test_'+pasture_type+'.csv', index = False)
print('R2')
display(df_R2_training_output)
df_R2_training_output.to_csv('r2_training_'+pasture_type+'.csv', index = False)
display(df_R2_test_output)
df_R2_test_output.to_csv('r2_test_'+pasture_type+'.csv', index = False)

RMSE


,Model,3D,5D,7D,10D,14D,30D,90D,180D,365D
0,XGBoost,297.530572,295.345652,294.410219,293.422940,292.107218,291.641031,291.537596,291.547860,291.536378
1,Bootstrap,295.990097,295.310457,294.013835,293.014517,292.069022,291.729432,291.625456,291.587935,291.589949
2,Random Forest,487.531290,474.077122,430.558076,419.822185,373.802468,319.280947,305.358054,303.396108,301.341354
3,Linear Regression,843.881527,814.774002,784.799936,748.129196,719.456119,599.951202,473.711338,437.322270,494.920065
4,DNN,299.740697,310.720164,314.670055,305.450780,316.206814,307.887731,298.143374,313.602535,317.785738
5,Stacking XGB DNN,295.712894,294.531139,293.900843,293.270171,292.063513,291.678110,291.927118,291.936407,291.863163
6,Stacking XGB RF,295.043023,295.069856,298.408749,294.069978,296.099818,292.516877,296.181019,296.959618,296.392702


,Model,3D,5D,7D,10D,14D,30D,90D,180D,365D
0,XGBoost,297.932161,296.223361,296.265601,295.230427,294.073484,293.271421,293.376225,293.209391,293.198980
1,Bootstrap,296.524955,295.454316,295.119395,294.315552,294.058337,293.343238,293.155347,293.087333,293.190211
2,Random Forest,476.584564,467.373161,427.311030,429.218302,382.889866,321.057491,301.441026,302.750013,301.490398
3,Linear Regression,836.496911,799.447145,771.806773,741.016137,710.883237,593.814149,470.304095,434.075883,488.654311
4,DNN,300.640089,312.868792,313.579293,306.292762,317.029083,307.578991,298.053055,308.205376,311.233482
5,Stacking XGB DNN,296.243683,295.853808,295.681170,295.079537,293.995427,293.120323,293.287594,293.164118,292.994052
6,Stacking XGB RF,296.414888,297.104369,301.240225,296.046778,298.858093,293.987071,297.543112,298.691720,297.909316


MAE


,Model,3D,5D,7D,10D,14D,30D,90D,180D,365D
0,XGBoost,181.307476,178.580491,177.559380,176.238432,174.386694,173.673304,173.544553,173.524837,173.548870
1,Bootstrap,178.672916,178.178467,176.616562,175.456132,173.962918,173.547745,173.444821,173.374287,173.415348
2,Random Forest,307.387711,308.990326,278.351213,267.356569,231.923516,199.866113,184.205622,187.756025,184.119639
3,Linear Regression,648.216008,625.868250,601.554605,564.095762,546.780957,435.044984,323.651494,324.474844,356.883457
4,DNN,182.632037,195.245100,196.378333,188.678561,194.673405,186.723909,182.028136,185.524685,190.014728
5,Stacking XGB DNN,179.047205,177.663710,177.054628,176.091780,174.332437,173.673192,174.147530,173.741225,173.874303
6,Stacking XGB RF,178.723229,177.313142,179.672104,176.449849,176.850874,174.344979,176.784430,176.721075,176.516513


,Model,3D,5D,7D,10D,14D,30D,90D,180D,365D
0,XGBoost,184.700092,182.581178,181.493048,180.539026,178.365725,177.479864,177.286721,177.390999,177.366852
1,Bootstrap,182.479808,181.718492,180.367301,179.116906,178.035007,177.387961,177.041667,177.068838,177.192327
2,Random Forest,309.385472,313.695016,282.878209,276.533778,242.557967,204.219115,184.298643,188.867898,186.192296
3,Linear Regression,651.017375,620.297680,599.431214,569.611656,550.031368,436.601430,329.677778,326.711472,358.014177
4,DNN,186.456736,198.319029,199.768921,192.156447,199.205876,189.698818,184.807283,185.627706,190.210278
5,Stacking XGB DNN,182.336139,181.658445,180.713725,180.168373,178.195690,177.370164,177.686050,177.377905,177.465739
6,Stacking XGB RF,182.329060,181.031723,183.539980,180.198058,180.748060,177.910692,179.676496,179.617968,179.486991


R2


,Model,3D,5D,7D,10D,14D,30D,90D,180D,365D
0,XGBoost,0.919127,0.920310,0.920814,0.921345,0.922048,0.922297,0.922352,0.922347,0.922353
1,Bootstrap,0.919962,0.920329,0.921027,0.921563,0.922069,0.922250,0.922305,0.922325,0.922324
2,Random Forest,0.782857,0.794676,0.830643,0.838983,0.872349,0.906871,0.914816,0.915907,0.917042
3,Linear Regression,0.349415,0.393522,0.437324,0.488678,0.527122,0.671169,0.794993,0.825279,0.776225
4,DNN,0.917921,0.911798,0.909541,0.914764,0.908655,0.913398,0.918793,0.910154,0.907741
5,Stacking XGB DNN,0.920112,0.920749,0.921088,0.921426,0.922072,0.922277,0.922144,0.922139,0.922179
6,Stacking XGB RF,0.920474,0.920459,0.918649,0.920997,0.919903,0.921830,0.919859,0.919437,0.919744


,Model,3D,5D,7D,10D,14D,30D,90D,180D,365D
0,XGBoost,0.918572,0.919504,0.919481,0.920042,0.920668,0.921100,0.921044,0.921133,0.921139
1,Bootstrap,0.919340,0.919921,0.920102,0.920537,0.920676,0.921061,0.921162,0.921199,0.921144
2,Random Forest,0.791638,0.799615,0.832496,0.830997,0.865511,0.905441,0.916643,0.915917,0.916616
3,Linear Regression,0.358101,0.413703,0.453544,0.496275,0.536410,0.676526,0.797094,0.827150,0.780951
4,DNN,0.917085,0.910203,0.909795,0.913938,0.907799,0.913214,0.918506,0.912860,0.911139
5,Stacking XGB DNN,0.919493,0.919704,0.919798,0.920124,0.920710,0.921181,0.921091,0.921158,0.921249
6,Stacking XGB RF,0.919399,0.919024,0.916754,0.919600,0.918065,0.920714,0.918785,0.918157,0.918585


----------------------------------------------------------------------------------------------------------------------